This notebook will show my work for the coding challenge


Challenge:

The Chief Information Security Officer wants to know that our external websites are properly secured and are functional for our clients. Please write a script to do the following:

 

    Retrieve up to date SSL scan results for www.elliottmgmt.com from Qualys SSL Labs API documentation is available at (https://github.com/ssllabs/ssllabs-scan/blob/stable/ssllabs-api-docs.md)
    Format relevant information into a report ready for email distribution.

 

Once you have completed your script, please create a Dockerfile so we can easily deploy and run it.

 



Let's look at that documentation

In [5]:
!curl -L https://github.com/ssllabs/ssllabs-scan/blob/stable/ssllabs-api-docs.md?raw=true

# SSL Labs API Documentation v1.24.4 #

**Last update:** 22 October 2016

This document explains the SSL Labs Assessment APIs, which can be used to test SSL servers available on the public Internet.

## Protocol Overview ##

The protocol is based on HTTP and JSON. All invocations of the API should use the GET method and specify the parameters in the query string, as documented below. The results will be returned in the response body as a JSON payload. In essence, the client submits an assessment request to the servers. If an acceptable report is already available, it's received straight away. Otherwise, the server will start a new assessment and the client should periodically check to see if the job is complete.

### Terms and Conditions ###

SSL Labs APIs are provided free of charge, subject to our terms and conditions: <https://www.ssllabs.com/about/terms.html>. The spirit of the license is that the APIs are made available so that system operators can test their own infrastructure. P

* **array[]** - an array

### Host ###

* **host** - assessment host, which can be a hostname or an IP address
* **port** - assessment port (e.g., 443)
* **protocol** - protocol (e.g., HTTP)
* **isPublic** - true if this assessment is publicly available (listed on the SSL Labs assessment boards)
* **status** - assessment status; possible values: DNS, ERROR, IN_PROGRESS, and READY.
* **statusMessage** - status message in English. When status is ERROR, this field will contain an error message.
* **startTime** - assessment starting time, in milliseconds since 1970
* **testTime** - assessment completion time, in milliseconds since 1970
* **engineVersion** - assessment engine version (e.g., "1.0.120")
* **criteriaVersion** - grading criteria version (e.g., "2009")
* **cacheExpiryTime** - when will the assessment results expire from the cache (typically set only for assessment with errors; otherwise the results stay in the cache for as long as there's sufficient room)
* **endpoints[]** - lis

Cool, now let's check that the API is up, and try analyzing  the requested page

In [6]:
!curl https://api.dev.ssllabs.com/api/v2/info

{"engineVersion":"2.1.11","criteriaVersion":"2009q","clientMaxAssessments":10,"maxAssessments":10,"currentAssessments":0,"newAssessmentCoolOff":1000,"messages":["This assessment service is provided free of charge by Qualys SSL Labs, subject to our terms and conditions: https://dev.ssllabs.com/about/terms.html"]}

In [9]:
!curl https://api.dev.ssllabs.com/api/v2/analyze?host=www.elliottmgmt.com|jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   378    0   378    0     0   1848      0 --:--:-- --:--:-- --:--:--  1852
{
  "host": "www.elliottmgmt.com",
  "port": 443,
  "protocol": "http",
  "isPublic": false,
  "status": "IN_PROGRESS",
  "startTime": 1699540211936,
  "engineVersion": "2.1.11",
  "criteriaVersion": "2009q",
  "endpoints": [
    {
      "ipAddress": "23.185.0.2",
      "statusMessage": "In progress",
      "statusDetails": "TESTING_HANDSHAKE_SIMULATION",
      "statusDetailsMessage": "Simulating handshakes",
      "progress": 86,
      "eta": 3,
      "delegation": 2
    }
  ]
}


So it needs to be called again times to give a final answer

In [10]:
!curl https://api.dev.ssllabs.com/api/v2/analyze?host=www.elliottmgmt.com|jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   393    0   393    0     0   1660      0 --:--:-- --:--:-- --:--:--  1665
{
  "host": "www.elliottmgmt.com",
  "port": 443,
  "protocol": "http",
  "isPublic": false,
  "status": "READY",
  "startTime": 1699540211936,
  "testTime": 1699540253518,
  "engineVersion": "2.1.11",
  "criteriaVersion": "2009q",
  "endpoints": [
    {
      "ipAddress": "23.185.0.2",
      "statusMessage": "Ready",
      "grade": "A",
      "gradeTrustIgnored": "A",
      "hasWarnings": false,
      "isExceptional": false,
      "progress": 100,
      "duration": 41571,
      "eta": 3,
      "delegation": 2
    }
  ]
}


Cool, now let's see the info for the particular endpoint

In [14]:
!curl https://api.dev.ssllabs.com/api/v2/getEndpointData?host=www.elliottmgmt.com\&s=23.185.0.2|jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19803    0 19803    0     0  53618      0 --:--:-- --:--:-- --:--:-- 53666
{
  "ipAddress": "23.185.0.2",
  "statusMessage": "Ready",
  "grade": "A",
  "gradeTrustIgnored": "A",
  "hasWarnings": false,
  "isExceptional": false,
  "progress": 100,
  "duration": 41571,
  "eta": 6,
  "delegation": 2,
  "details": {
    "hostStartTime": 1699540211936,
    "key": {
      "size": 2048,
      "alg": "RSA",
      "debianFlaw": false,
      "strength": 2048
    },
    "cert": {
      "subject": "CN=elliottmgmt.com",
      "commonNames": [
        "elliottmgmt.com"
      ],
      "altNames": [
        "elliottmgmt.com",
        "www.elliottmgmt.com"
      ],
      "notBefore": 1696198605000,
      "notAfter": 1703974604000,
      "issuerSubject": "CN=R3, O=Let's Encrypt, C=US",
      "issuerLabel": "R3",
      "sigAlg": "SHA256withRSA",

Cool, that's a lot of relevant data...
To generate the report we can use Python.
We can generate a Markdown document and format it as PDF via pandoc.
We can structure the report as follows:
## Qualys Reachability and Security report


### Overview
|Key|Value|
| :-- | :-- |
|Domain| www.elliottmgmt.com|
|Report time| (startDate)
|(Worst) Grade| A
|(Wost) Grade (ignoring certificate trust)| A
|Earliest certificate expiration|(notAfter)
|Vulnerabilities detected| []|
HSTS status:s

### Endpoints

| IP | Grade | Grade ignoring trust| expiration | heartbleed | heartbeat | poodle | logjam | drown|
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 23.185.0.2 |A |A |||||||
| ip2... | | |||||||

### Client

|Platform| ip1 (23.185.0.2)| (ip2)|... |
| --- | --- | --- | --- |
|Safari OS X 10.11| Success | ...|
|Win8 v1 IE|<span style="background:pink">Fail</span>|... |...|
|...| ... |... |...|
#### Error Codes

|Platform|IP|attempts|error code|
| --- | --- | --- | --- | 
|Win8 v1 IE|23.185.0.2|1|1|
| ...|||

So now let's do this in Python

In [41]:
import requests
import json
from time import sleep
from pprint import pprint
from datetime import datetime

In [30]:
def get_initial_info(site, debug=False):
    status = ''
    base_url=f'https://api.dev.ssllabs.com/api/v2/analyze?host={site}'
    if debug:
        url=f'{base_url}&startNew=on'
    else:
        url=base_url
    while (response:=requests.get(url).json())['status'] !='READY':
        url=base_url
        if debug:
            pprint(response)
        sleep(max([2]+[e.get('eta',2) for e in response.get('endpoints',[])]))
    return response

pprint(get_initial_info('www.elliottmgmt.com',debug=True))
pprint(get_initial_info('google.com',debug=True))
pprint(get_initial_info('en.wikipedia.org',debug=True))

{'criteriaVersion': '2009q',
 'endpoints': [{'delegation': 2,
                'eta': 5,
                'ipAddress': '23.185.0.2',
                'progress': 83,
                'statusDetails': 'TESTING_SUITES',
                'statusDetailsMessage': 'Determining available cipher suites',
                'statusMessage': 'In progress'}],
 'engineVersion': '2.1.11',
 'host': 'www.elliottmgmt.com',
 'isPublic': False,
 'port': 443,
 'protocol': 'http',
 'startTime': 1699579031346,
 'status': 'IN_PROGRESS'}
{'criteriaVersion': '2009q',
 'endpoints': [{'delegation': 2,
                'eta': 5,
                'ipAddress': '23.185.0.2',
                'progress': 85,
                'statusDetails': 'TESTING_HANDSHAKE_SIMULATION',
                'statusDetailsMessage': 'Simulating handshakes',
                'statusMessage': 'In progress'}],
 'engineVersion': '2.1.11',
 'host': 'www.elliottmgmt.com',
 'isPublic': False,
 'port': 443,
 'protocol': 'http',
 'startTime': 1699579031346,


{'criteriaVersion': '2009q',
 'endpoints': [{'delegation': 1,
                'duration': 101193,
                'eta': 1,
                'grade': 'B',
                'gradeTrustIgnored': 'B',
                'hasWarnings': False,
                'ipAddress': '142.250.191.238',
                'isExceptional': False,
                'progress': 100,
                'serverName': 'ord38s32-in-f14.1e100.net',
                'statusMessage': 'Ready'},
               {'delegation': 1,
                'duration': 101196,
                'eta': 1,
                'grade': 'B',
                'gradeTrustIgnored': 'B',
                'hasWarnings': False,
                'ipAddress': '2607:f8b0:4009:80b:0:0:0:200e',
                'isExceptional': False,
                'progress': 100,
                'serverName': 'ord37s35-in-x0e.1e100.net',
                'statusMessage': 'Ready'}],
 'engineVersion': '2.1.11',
 'host': 'google.com',
 'isPublic': False,
 'port': 443,
 'protocol': 'h

{'criteriaVersion': '2009q',
 'endpoints': [{'delegation': 1,
                'duration': 46400,
                'eta': 4,
                'grade': 'A+',
                'gradeTrustIgnored': 'A+',
                'hasWarnings': False,
                'ipAddress': '208.80.153.224',
                'isExceptional': True,
                'progress': 100,
                'serverName': 'text-lb.codfw.wikimedia.org',
                'statusMessage': 'Ready'},
               {'delegation': 1,
                'duration': 46481,
                'eta': 2,
                'grade': 'A+',
                'gradeTrustIgnored': 'A+',
                'hasWarnings': False,
                'ipAddress': '2620:0:860:ed1a:0:0:0:1',
                'isExceptional': True,
                'progress': 100,
                'serverName': 'text-lb.codfw.wikimedia.org',
                'statusMessage': 'Ready'}],
 'engineVersion': '2.1.11',
 'host': 'en.wikipedia.org',
 'isPublic': False,
 'port': 443,
 'protocol':

In [48]:
def get_endpoint_info(site):
    initial_info = get_initial_info(site)
    endpoint_infos = [
        requests.get(f'https://api.dev.ssllabs.com/api/v2/getEndpointData?host={site}&s={endpoint["ipAddress"]}').json() 
        for endpoint in initial_info.get('endpoints',[])
    ]
    return endpoint_infos
    
pprint(get_endpoint_info('www.elliottmgmt.com'))
pprint(get_endpoint_info('google.com'))
pprint(get_endpoint_info('en.wikipedia.org'))

[{'delegation': 2,
  'details': {'cert': {'altNames': ['elliottmgmt.com', 'www.elliottmgmt.com'],
                       'commonNames': ['elliottmgmt.com'],
                       'crlRevocationStatus': 4,
                       'crlURIs': [''],
                       'issuerLabel': 'R3',
                       'issuerSubject': "CN=R3, O=Let's Encrypt, C=US",
                       'issues': 0,
                       'mustStaple': 0,
                       'notAfter': 1703974604000,
                       'notBefore': 1696198605000,
                       'ocspRevocationStatus': 2,
                       'ocspURIs': ['http://r3.o.lencr.org'],
                       'pinSha256': 'tb0gpmFSHp26fT3bLpiBcyAYMke2WdQY0e4cnIlWi4E=',
                       'revocationInfo': 2,
                       'revocationStatus': 2,
                       'sct': True,
                       'sgc': 0,
                       'sha1Hash': 'eb2b51bd727f62651be647abc0b32373ea3dadc8',
                       'sig

[{'delegation': 1,
  'details': {'cert': {'altNames': ['*.google.com',
                                    '*.appengine.google.com',
                                    '*.bdn.dev',
                                    '*.origin-test.bdn.dev',
                                    '*.cloud.google.com',
                                    '*.crowdsource.google.com',
                                    '*.datacompute.google.com',
                                    '*.google.ca',
                                    '*.google.cl',
                                    '*.google.co.in',
                                    '*.google.co.jp',
                                    '*.google.co.uk',
                                    '*.google.com.ar',
                                    '*.google.com.au',
                                    '*.google.com.br',
                                    '*.google.com.co',
                                    '*.google.com.mx',
                                

[{'delegation': 1,
  'details': {'cert': {'altNames': ['*.m.mediawiki.org',
                                    '*.m.wikibooks.org',
                                    '*.m.wikidata.org',
                                    '*.m.wikimedia.org',
                                    '*.m.wikinews.org',
                                    '*.m.wikipedia.org',
                                    '*.m.wikiquote.org',
                                    '*.m.wikisource.org',
                                    '*.m.wikiversity.org',
                                    '*.m.wikivoyage.org',
                                    '*.m.wiktionary.org',
                                    '*.mediawiki.org',
                                    '*.planet.wikimedia.org',
                                    '*.wikibooks.org',
                                    '*.wikidata.org',
                                    '*.wikifunctions.org',
                                    '*.wikimedia.org',
           

In [39]:
sorted(['A,','A+,','A-,','B']) # showing that adding a comma makes it sort correctly

['A+,', 'A,', 'A-,', 'B']

In [38]:
print([chr(i) for i in range(ord('+'),ord('-'))]) # because comma is between plus and minus

['+', ',']


In [98]:
from IPython.display import display_markdown
def generate_markdown_report(site, filename):
    info=get_initial_info(site)
    endpoints=get_endpoint_info(site)
    clients={s['client']['id']: s for e in endpoints for s in e['details']['sims']['results']}
    sims = {client_id:{e['ipAddress']:sim 
                                for e in endpoints  
                                for sim in  e['details']['sims']['results']
                                if sim['client']['id']==client_id } 
                     for client_id in clients.keys()}
    report = f'''### Overview
|Key|Value|
| :-- | :-- |
|Domain| {site}|
|Report time| {datetime.utcfromtimestamp(info['startTime']//1000).strftime("%FT%T")} UTC
|(Worst) Grade| {min([f'{e["grade"]},' for e in endpoints])[:-1]}
|(Worst) Grade (ignoring certificate trust)| {min([f'{e["gradeTrustIgnored"]},' for e in endpoints])[:-1]}
|Earliest certificate expiration|{datetime.utcfromtimestamp(min([e['details']['cert']['notAfter']//1000 for e in endpoints])).strftime('%FT%T')}
|Vulnerabilities detected| {[vuln for vuln in ['heartbleed','heartbeat','freak','poodle','logjam'] if any(e.get(vuln) for e in endpoints)]}|

### Endpoints

| IP | Grade | Grade ignoring trust| expiration | heartbleed | heartbeat | poodle | logjam | drown|
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
{chr(10).join(f'| {e["ipAddress"]} | {e["grade"]} | {e["gradeTrustIgnored"]} |' 
f'{datetime.utcfromtimestamp(e["details"]["cert"]["notAfter"]//1000)}|{"|".join(str(e["details"][vuln]) for vuln in ("heartbleed", "heartbeat","freak","poodle","logjam"))}|' 
for e in endpoints)}

### Client

|Client ID| Client Name| Client Version|Platform|{"|".join(e["ipAddress"] for e in endpoints)}|
| --- | --- | --- | --- | --- |
{chr(10).join(
    f"""|{client_id}|{(
    clients[client_id]['client'].get('name',''))}|{(
    clients[client_id]['client'].get('version',''))}|{(
    clients[client_id]['client'].get('platform',''))}|{(
    '|'.join((
        '<span style="background:pink">Fail</span>' 
        if sims[client_id].get(e['ipAddress'],{}).get('attempts',0) 
            and sims[client_id].get(e['ipAddress'],{}).get('errorCode',0) 
        else '<span style="background:green">Success</span>') 
    for e in endpoints))}""" for client_id in list(clients.keys()))}
'''
    with open(filename,'w') as f:
        f.write(report)
filename='www.elliottmgmt.com.md'
generate_markdown_report('www.elliottmgmt.com',filename)
md=open(filename).read()

display_markdown(md,raw=True)

### Overview
|Key|Value|
| :-- | :-- |
|Domain| www.elliottmgmt.com|
|Report time| 2023-11-10T03:22:36 UTC
|(Worst) Grade| A
|(Worst) Grade (ignoring certificate trust)| A
|Earliest certificate expiration|2023-12-30T22:16:44
|Vulnerabilities detected| []|

### Endpoints

| IP | Grade | Grade ignoring trust| expiration | heartbleed | heartbeat | poodle | logjam | drown|
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 23.185.0.2 | A | A |2023-12-30 22:16:44|False|False|False|False|False|

### Client

|Client ID| Client Name| Client Version|Platform|23.185.0.2|
| --- | --- | --- | --- | --- |
|56|Android|2.3.7||<span style="background:green">Success</span>
|58|Android|4.0.4||<span style="background:green">Success</span>
|59|Android|4.1.1||<span style="background:green">Success</span>
|60|Android|4.2.2||<span style="background:green">Success</span>
|61|Android|4.3||<span style="background:green">Success</span>
|62|Android|4.4.2||<span style="background:green">Success</span>
|88|Android|5.0.0||<span style="background:green">Success</span>
|129|Android|6.0||<span style="background:green">Success</span>
|167|Android|7.0||<span style="background:green">Success</span>
|168|Android|8.0||<span style="background:green">Success</span>
|157|Android|8.1||<span style="background:green">Success</span>
|158|Android|9.0||<span style="background:green">Success</span>
|94|Baidu|Jan 2015||<span style="background:green">Success</span>
|91|BingPreview|Jan 2015||<span style="background:green">Success</span>
|136|Chrome|49|XP SP3|<span style="background:green">Success</span>
|152|Chrome|69|Win 7|<span style="background:green">Success</span>
|153|Chrome|70|Win 10|<span style="background:green">Success</span>
|170|Chrome|80|Win 10|<span style="background:green">Success</span>
|84|Firefox|31.3.0 ESR|Win 7|<span style="background:green">Success</span>
|132|Firefox|47|Win 7|<span style="background:green">Success</span>
|137|Firefox|49|XP SP3|<span style="background:green">Success</span>
|151|Firefox|62|Win 7|<span style="background:green">Success</span>
|171|Firefox|73|Win 10|<span style="background:green">Success</span>
|145|Googlebot|Feb 2018||<span style="background:green">Success</span>
|100|IE|6|XP|<span style="background:green">Success</span>
|19|IE|7|Vista|<span style="background:green">Success</span>
|101|IE|8|XP|<span style="background:green">Success</span>
|113|IE|8-10|Win 7|<span style="background:green">Success</span>
|143|IE|11|Win 7|<span style="background:pink">Fail</span>
|134|IE|11|Win 8.1|<span style="background:pink">Fail</span>
|64|IE|10|Win Phone 8.0|<span style="background:green">Success</span>
|65|IE|11|Win Phone 8.1|<span style="background:pink">Fail</span>
|106|IE|11|Win Phone 8.1 Update|<span style="background:pink">Fail</span>
|131|IE|11|Win 10|<span style="background:green">Success</span>
|144|Edge|15|Win 10|<span style="background:green">Success</span>
|159|Edge|16|Win 10|<span style="background:green">Success</span>
|160|Edge|18|Win 10|<span style="background:green">Success</span>
|120|Edge|13|Win Phone 10|<span style="background:green">Success</span>
|25|Java|6u45||<span style="background:green">Success</span>
|26|Java|7u25||<span style="background:green">Success</span>
|147|Java|8u161||<span style="background:green">Success</span>
|162|Java|11.0.3||<span style="background:green">Success</span>
|163|Java|12.0.1||<span style="background:green">Success</span>
|27|OpenSSL|0.9.8y||<span style="background:green">Success</span>
|99|OpenSSL|1.0.1l||<span style="background:green">Success</span>
|164|OpenSSL|1.0.2s||<span style="background:green">Success</span>
|169|OpenSSL|1.1.0k||<span style="background:green">Success</span>
|165|OpenSSL|1.1.1c||<span style="background:green">Success</span>
|32|Safari|5.1.9|OS X 10.6.8|<span style="background:green">Success</span>
|33|Safari|6|iOS 6.0.1|<span style="background:pink">Fail</span>
|34|Safari|6.0.4|OS X 10.8.4|<span style="background:green">Success</span>
|63|Safari|7|iOS 7.1|<span style="background:pink">Fail</span>
|35|Safari|7|OS X 10.9|<span style="background:pink">Fail</span>
|85|Safari|8|iOS 8.4|<span style="background:pink">Fail</span>
|87|Safari|8|OS X 10.10|<span style="background:pink">Fail</span>
|114|Safari|9|iOS 9|<span style="background:green">Success</span>
|111|Safari|9|OS X 10.11|<span style="background:green">Success</span>
|140|Safari|10|iOS 10|<span style="background:green">Success</span>
|138|Safari|10|OS X 10.12|<span style="background:green">Success</span>
|161|Safari|12.1.2|MacOS 10.14.6 Beta|<span style="background:green">Success</span>
|166|Safari|12.1.1|iOS 12.3.1|<span style="background:green">Success</span>
|112|Apple ATS|9|iOS 9|<span style="background:green">Success</span>
|92|Yahoo Slurp|Jan 2015||<span style="background:green">Success</span>
|93|YandexBot|Jan 2015||<span style="background:green">Success</span>


Output looks reasonable, let's create a report in HTML and PDF

In [6]:
from IPython.display import IFrame

In [5]:
!pandoc -f markdown {filename} -t html > {filename}.html

IFrame(f"{filename}.html", width=1200, height=800)

NameError: name 'IFrame' is not defined

In [111]:
!pandoc -f markdown {filename} -t pdf > {filename}.pdf
IFrame(f"{filename}.pdf", width=1500, height=800)

In [114]:
# Hmm, those tables are smushed, let's try reducing margins
filename='www.elliottmgmt.com2.md'
generate_markdown_report('www.elliottmgmt.com',filename)
header='''
---
title:  Qualys Report
author: oz
geometry: margin=1cm
---
'''
md=header+open(filename).read()
open(filename,'w').write(md)

!pandoc -f markdown {filename} -t pdf > {filename}.pdf
IFrame(f"{filename}.pdf", width=1500, height=800)

Alright, now let's save the script and dockerize

In [42]:
%%bash
cat > main.py <<EOF
import requests
import json
from time import sleep
from pprint import pprint
from datetime import datetime
import click
from subprocess import check_call
import tempfile
def get_initial_info(site, debug=False):
    status = ''
    base_url=f'https://api.dev.ssllabs.com/api/v2/analyze?host={site}'
    if debug:
        url=f'{base_url}&startNew=on'
    else:
        url=base_url
    while (response:=requests.get(url).json())['status'] !='READY':
        url=base_url
        if debug:
            pprint(response)
        sleep(max([2]+[e.get('eta',2) for e in response.get('endpoints',[])]))
    return response
def get_endpoint_info(site):
    initial_info = get_initial_info(site)
    endpoint_infos = [
        requests.get(f'https://api.dev.ssllabs.com/api/v2/getEndpointData?host={site}&s={endpoint["ipAddress"]}').json() 
        for endpoint in initial_info.get('endpoints',[])
    ]
    return endpoint_infos

@click.command()
@click.option('--site', help='domain for which to generate report.')
@click.option('--filename', help='Where to save the report; end with .md, .html, or .pdf to get the appropriate file type')
def generate_markdown_report(site, filename):
    '''Script to generate Qualys report'''
    info=get_initial_info(site)
    endpoints=get_endpoint_info(site)
    clients={s['client']['id']: s for e in endpoints for s in e['details']['sims']['results']}
    sims = {client_id:{e['ipAddress']:sim 
                                for e in endpoints  
                                for sim in  e['details']['sims']['results']
                                if sim['client']['id']==client_id } 
                     for client_id in clients.keys()}
    report = f'''---
title:  Qualys Report
author: oz
geometry: margin=1cm
---
### Overview
|Key|Value|
| :-- | :-- |
|Domain| {site}|
|Report time| {datetime.utcfromtimestamp(info['startTime']//1000).strftime("%FT%T")} UTC
|(Worst) Grade| {min([f'{e["grade"]},' for e in endpoints])[:-1]}
|(Worst) Grade (ignoring certificate trust)| {min([f'{e["gradeTrustIgnored"]},' for e in endpoints])[:-1]}
|Earliest certificate expiration|{datetime.utcfromtimestamp(min([e['details']['cert']['notAfter']//1000 for e in endpoints])).strftime('%FT%T')}
|Vulnerabilities detected| {[vuln for vuln in ['heartbleed','heartbeat','freak','poodle','logjam'] if any(e.get(vuln) for e in endpoints)]}|

### Endpoints

| IP | Grade | Grade ignoring trust| expiration | heartbleed | heartbeat | poodle | logjam | drown|
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
{chr(10).join(f'| {e["ipAddress"]} | {e["grade"]} | {e["gradeTrustIgnored"]} |' 
f'{datetime.utcfromtimestamp(e["details"]["cert"]["notAfter"]//1000)}|{"|".join(str(e["details"][vuln]) for vuln in ("heartbleed", "heartbeat","freak","poodle","logjam"))}|' 
for e in endpoints)}

### Client

|Client ID| Client Name| Client Version|Platform|{"|".join(e["ipAddress"] for e in endpoints)}|
| --- | --- | --- | --- | --- |
{chr(10).join(
    f"""|{client_id}|{(
    clients[client_id]['client'].get('name',''))}|{(
    clients[client_id]['client'].get('version',''))}|{(
    clients[client_id]['client'].get('platform',''))}|{(
    '|'.join((
        '<span style="background:pink">Fail</span>' 
        if sims[client_id].get(e['ipAddress'],{}).get('attempts',0) 
            and sims[client_id].get(e['ipAddress'],{}).get('errorCode',0) 
        else '<span style="background:green">Success</span>') 
    for e in endpoints))}""" for client_id in list(clients.keys()))}
'''
    print(filename)
    if filename[-3:]=='.md':
        with open(filename,'w') as f:
            f.write(report)
    else:
        with tempfile.NamedTemporaryFile(mode='w',delete=False) as tmp:
            tmp.write(report)
            tmp.file.close()
            if filename[-4:]=='.pdf':
                
                open(filename,'w').write(md)
                check_call(f'/usr/bin/pandoc -f markdown {tmp.name} -t pdf -o {filename}'.split())
            elif filename[-5:]=='.html':
                check_call(f'/usr/bin/pandoc -f markdown {tmp.name} -t html -o {filename}'.split())
if __name__ == '__main__':
    generate_markdown_report()
EOF

python3 main.py --help

Usage: main.py [OPTIONS]

  Script to generate Qualys report

Options:
  --site TEXT      domain for which to generate report.
  --filename TEXT  Where to save the report; end with .md, .html, or .pdf to
                   get the appropriate file type
  --help           Show this message and exit.


In [43]:
!python3 main.py --site www.elliottmgmt.com --filename report.html
IFrame(f"report.html", width=1500, height=800)

report.html


In [39]:
%%bash
# alright, now a dockerfile
cat > Dockerfile <<EOF
FROM ubuntu:22.04
RUN apt-get update && apt-get install python3-pip pandoc curl -y
RUN pip install requests click # TODO: pin version
RUN apt-get install texlive-latex-base texlive-latex-recommended texlive-fonts-recommended -y # TODO: move up
COPY main.py /main.py
CMD ["python3", "/main.py"]
EOF
docker build -t qualys-scan .

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/



Sending build context to Docker daemon  798.2kB
Step 1/6 : FROM ubuntu:22.04
 ---> e4c58958181a
Step 2/6 : RUN apt-get update && apt-get install python3-pip pandoc curl -y
 ---> Using cache
 ---> 5ee24df15e3c
Step 3/6 : RUN pip install requests click # TODO: pin version
 ---> Using cache
 ---> 6f0e682fa613
Step 4/6 : RUN apt-get install texlive-latex-base texlive-latex-recommended texlive-fonts-recommended -y # TODO: move up
 ---> Running in 6c2aca591cb5
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core dbus dbus-user-session
  dconf-gsettings-backend dconf-service dmsetup dvisvgm fontconfig
  fonts-droid-fallback fonts-lmodern fonts-noto-mono fonts-texgyre
  fonts-urw-base35 gir1.2-glib-2.0 gsettings-desktop-schemas
  gtk-update-icon-cache hicolor-icon-theme humanity-icon-theme libapparmor1
  libargon2-1 libatk-bridge2.0-0 libatk1.0-0 libatk1.0-data libatspi2.0-0
  li

Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libsystemd0 amd64 249.11-0ubuntu3.11 [318 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libapparmor1 amd64 3.0.4-2ubuntu2.2 [39.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libargon2-1 amd64 0~20171227-0.3 [19.5 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libdevmapper1.02.1 amd64 2:1.02.175-2.1ubuntu4 [139 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libjson-c5 amd64 0.15-3~ubuntu1.22.04.2 [33.5 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libcryptsetup12 amd64 2:2.4.3-1ubuntu1.1 [211 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libip4tc2 amd64 1.8.7-1ubuntu5.1 [19.8 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/main amd64 libkmod2 amd64 29-1ubuntu1 [48.0 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 systemd amd64 249.11-0ubuntu3.11 [4581 kB]
Get:10 http://archive.ubuntu.com/u

Get:80 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpixman-1-0 amd64 0.40.0-1ubuntu0.22.04.1 [264 kB]
Get:81 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-render0 amd64 1.14-3ubuntu3 [16.4 kB]
Get:82 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-shm0 amd64 1.14-3ubuntu3 [5780 B]
Get:83 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxrender1 amd64 1:0.9.10-1build4 [19.7 kB]
Get:84 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcairo2 amd64 1.16.0-5ubuntu2 [628 kB]
Get:85 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcairo-gobject2 amd64 1.16.0-5ubuntu2 [19.4 kB]
Get:86 http://archive.ubuntu.com/ubuntu jammy/main amd64 libclone-perl amd64 0.45-1build3 [11.0 kB]
Get:87 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblcms2-2 amd64 2.12~rc1-2build2 [159 kB]
Get:88 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolord2 amd64 1.4.6-1 [155 kB]
Get:89 http://archive.ubuntu.com/ubuntu jammy/main amd64 libdata-dump-perl all 1.25-

Get:158 http://archive.ubuntu.com/ubuntu jammy/main amd64 libhttp-cookies-perl all 6.10-1 [18.4 kB]
Get:159 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libhttp-daemon-perl all 6.13-1ubuntu0.1 [22.9 kB]
Get:160 http://archive.ubuntu.com/ubuntu jammy/main amd64 libhttp-negotiate-perl all 6.01-1 [12.5 kB]
Get:161 http://archive.ubuntu.com/ubuntu jammy/main amd64 libice6 amd64 2:1.0.10-1build2 [42.6 kB]
Get:162 http://archive.ubuntu.com/ubuntu jammy/main amd64 perl-openssl-defaults amd64 5build2 [7542 B]
Get:163 http://archive.ubuntu.com/ubuntu jammy/main amd64 libnet-ssleay-perl amd64 1.92-1build2 [327 kB]
Get:164 http://archive.ubuntu.com/ubuntu jammy/main amd64 libio-socket-ssl-perl all 2.074-2 [192 kB]
Get:165 http://archive.ubuntu.com/ubuntu jammy/main amd64 libio-stringy-perl all 2.111-3 [55.8 kB]
Get:166 http://archive.ubuntu.com/ubuntu jammy/main amd64 libnet-http-perl all 6.22-1 [23.2 kB]
Get:167 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtry-tiny-perl all 

...

Preparing to unpack .../085-libdrm-radeon1_2.4.113-2~ubuntu0.22.04.1_amd64.deb ...
Unpacking libdrm-radeon1:amd64 (2.4.113-2~ubuntu0.22.04.1) ...
Selecting previously unselected package libencode-locale-perl.
Preparing to unpack .../086-libencode-locale-perl_1.05-1.1_all.deb ...
Unpacking libencode-locale-perl (1.05-1.1) ...
Selecting previously unselected package libepoxy0:amd64.
Preparing to unpack .../087-libepoxy0_1.5.10-1_amd64.deb ...
Unpacking libepoxy0:amd64 (1.5.10-1) ...
Selecting previously unselected package libipc-system-simple-perl.
Preparing to unpack .../088-libipc-system-simple-perl_1.30-1_all.deb ...
Unpacking libipc-system-simple-perl (1.30-1) ...
Selecting previously unselected package libfile-basedir-perl.
Preparing to unpack .../089-libfile-basedir-perl_0.09-1_all.deb ...
Unpacking libfile-basedir-perl (0.09-1) ...
Selecting previously unselected package liburi-perl.
Preparing to unpack .../090-liburi-perl_5.10-1_all.deb ...
Unpacking liburi-perl (5.10-1) ...
Sele

Unpacking libxdamage1:amd64 (1:1.1.5-2build2) ...
Selecting previously unselected package libxinerama1:amd64.
Preparing to unpack .../130-libxinerama1_2%3a1.1.4-3_amd64.deb ...
Unpacking libxinerama1:amd64 (2:1.1.4-3) ...
Selecting previously unselected package libxkbcommon0:amd64.
Preparing to unpack .../131-libxkbcommon0_1.4.0-1_amd64.deb ...
Unpacking libxkbcommon0:amd64 (1.4.0-1) ...
Selecting previously unselected package libxrandr2:amd64.
Preparing to unpack .../132-libxrandr2_2%3a1.5.2-1build1_amd64.deb ...
Unpacking libxrandr2:amd64 (2:1.5.2-1build1) ...
Selecting previously unselected package libgtk-3-common.
Preparing to unpack .../133-libgtk-3-common_3.24.33-1ubuntu2_all.deb ...
Unpacking libgtk-3-common (3.24.33-1ubuntu2) ...
Selecting previously unselected package libgtk-3-0:amd64.
Preparing to unpack .../134-libgtk-3-0_3.24.33-1ubuntu2_amd64.deb ...
Unpacking libgtk-3-0:amd64 (3.24.33-1ubuntu2) ...
Selecting previously unselected package libgtk-3-bin.
Preparing to unpack 

Preparing to unpack .../175-libunwind8_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind8:amd64 (1.3.2-2build2.1) ...
Selecting previously unselected package libvte-2.91-common.
Preparing to unpack .../176-libvte-2.91-common_0.68.0-1_amd64.deb ...
Unpacking libvte-2.91-common (0.68.0-1) ...
Selecting previously unselected package libvte-2.91-0:amd64.
Preparing to unpack .../177-libvte-2.91-0_0.68.0-1_amd64.deb ...
Unpacking libvte-2.91-0:amd64 (0.68.0-1) ...
Selecting previously unselected package libvted-3-0:amd64.
Preparing to unpack .../178-libvted-3-0_3.10.0-1ubuntu1_amd64.deb ...
Unpacking libvted-3-0:amd64 (3.10.0-1ubuntu1) ...
Selecting previously unselected package libx11-protocol-perl.
Preparing to unpack .../179-libx11-protocol-perl_0.56-7.1_all.deb ...
Unpacking libx11-protocol-perl (0.56-7.1) ...
Selecting previously unselected package libxt6:amd64.
Preparing to unpack .../180-libxt6_1%3a1.2.1-1_amd64.deb ...
Unpacking libxt6:amd64 (1:1.2.1-1) ...
Selecting previously unsel

invoke-rc.d: policy-rc.d denied execution of start.
Setting up libtry-tiny-perl (0.31-1) ...
Setting up libsensors-config (1:3.6.0-7ubuntu1) ...
Setting up perl-openssl-defaults:amd64 (5build2) ...
Setting up libxext6:amd64 (2:1.3.4-1build1) ...
Setting up libnss-systemd:amd64 (249.11-0ubuntu3.11) ...
First installation detected...
Checking NSS setup...
Setting up xkb-data (2.33-1) ...
Setting up libencode-locale-perl (1.05-1.1) ...
Setting up libxcb-shm0:amd64 (1.14-3ubuntu3) ...
Setting up libzzip-0-13:amd64 (0.13.72+dfsg.1-1.1) ...
Setting up libpaper-utils (1.1.28build2) ...
Setting up fonts-urw-base35 (20200910-1) ...
Setting up libunwind8:amd64 (1.3.2-2build2.1) ...
Setting up libcairo2:amd64 (1.16.0-5ubuntu2) ...
Setting up libcolord2:amd64 (1.4.6-1) ...
Setting up libxxf86vm1:amd64 (1:1.1.4-1build3) ...
Setting up poppler-data (0.4.11-1) ...
Setting up libxcb-present0:amd64 (1.14-3ubuntu3) ...
Setting up libdconf1:amd64 (0.40.0-3) ...
Setting up tex-common (6.17) ...
debconf: u

Setting up dvisvgm (2.13.1-1) ...
Setting up libxaw7:amd64 (2:1.0.14-1) ...
Setting up x11-xserver-utils (7.7+9build1) ...
Setting up librsvg2-2:amd64 (2.52.5+dfsg-3ubuntu0.2) ...
Setting up librsvg2-common:amd64 (2.52.5+dfsg-3ubuntu0.2) ...
Setting up texlive-binaries (2021.20210626.59705-1ubuntu0.1) ...
update-alternatives: using /usr/bin/xdvi-xaw to provide /usr/bin/xdvi.bin (xdvi.bin) in auto mode
update-alternatives: using /usr/bin/bibtex.original to provide /usr/bin/bibtex (bibtex) in auto mode
update-alternatives: warning: skip creation of /usr/share/man/man1/bibtex.1.gz because associated file /usr/share/man/man1/bibtex.original.1.gz (of link group bibtex) doesn't exist
Setting up libgdk-pixbuf2.0-bin (2.42.8+dfsg-1ubuntu0.2) ...
Setting up texlive-base (2021.20220204-1) ...
/usr/bin/ucfr
/usr/bin/ucfr
/usr/bin/ucfr
/usr/bin/ucfr
tl-paper: setting paper size for dvips to a4: /var/lib/texmf/dvips/config/config-paper.ps
tl-paper: setting paper size for dvipdfmx to a4: /var/lib/te

In [40]:
!docker run -it -v `pwd`:/work qualys-scan python3 /main.py --filename work/output.pdf --site www.elliottmgmt.com

work/output.pdf


In [41]:
IFrame(f"output.pdf", width=1500, height=800)

Assuming you have coded this challenge, now answer the following questions in a few sentences within the readme:

    How would you scale this script and run it with resiliency to e.g. handle 1000s of domains?
    How would you monitor/alert on this service?
    What would you do to handle adding new domains to scan or certificate expiry events from your service?
    After some time, your report requires more enhancements requested by the Tech team of the company. How would you handle these "continuous" requirement changes in a sustainable manner?

In [52]:
%%bash

cat >README.md <<EOF
# Qualys Scanner report (Elliott Management Interview)

The Python script in main.py calls the Qualys API to scan a site and produce a report; the report is output using Markdown for simplicity, and then converted using Pandoc to PDF or HTML.

The process for writing the script is detailed in Notebook.ipynb.
Note the script is not fully production ready, eg there is no handling of rate limit and server down conditions

## Q&A
### How would you scale this script and run it with resiliency to e.g. handle 1000s of domains?
For production use, the domains should come from an API endpoint or queue (eg Kafka, SQS). The service can be run on Kubernetes as a Deployment, with a Service for load balancing and an Ingress/Gateway for routing and TLS termination (if relevant). HPA can be set up to scale on queue depth

Note also that use at scale would require an agreement with Qualys to bypass the rate limit
### How would you monitor/alert on this service?
You would want to alert on:
- Qualys /info endpoint responding
- checks to a /health endpoint to make sure the system is responding
- occasional requests for a particular domain to see that the process as a whole is still working. Make sure Qualys is returning cached data for this domain, so you can measure latency.
### What would you do to handle adding new domains to scan or certificate expiry events from your service?
See above, requests should come in via HTTP or a queue
### After some time, your report requires more enhancements requested by the Tech team of the company. How would you handle these "continuous" requirement changes in a sustainable manner?
The code is kept in a git or similar repo; a deployment process is set up (eg with Jenkins or Github actions) that can deploy new versions of the service; staging and dev environments are set up for validating changes; unit and integration tests are added to the code; 
EOF

In [53]:
display_markdown(open("README.md").read(),raw=True)

# Qualys Scanner report (Elliott Management Interview)

The Python script in main.py calls the Qualys API to scan a site and produce a report; the report is output using Markdown for simplicity, and then converted using Pandoc to PDF or HTML.

The process for writing the script is detailed in Notebook.ipynb.
Note the script is not fully production ready, eg there is no handling of rate limit and server down conditions

## Q&A
### How would you scale this script and run it with resiliency to e.g. handle 1000s of domains?
For production use, the domains should come from an API endpoint or queue (eg Kafka, SQS). The service can be run on Kubernetes as a Deployment, with a Service for load balancing and an Ingress/Gateway for routing and TLS termination (if relevant). HPA can be set up to scale on queue depth

Note also that use at scale would require an agreement with Qualys to bypass the rate limit
### How would you monitor/alert on this service?
You would want to alert on:
- Qualys /info endpoint responding
- checks to a /health endpoint to make sure the system is responding
- occasional requests for a particular domain to see that the process as a whole is still working. Make sure Qualys is returning cached data for this domain, so you can measure latency.
### What would you do to handle adding new domains to scan or certificate expiry events from your service?
See above, requests should come in via HTTP or a queue
### After some time, your report requires more enhancements requested by the Tech team of the company. How would you handle these "continuous" requirement changes in a sustainable manner?
The code is kept in a git or similar repo; a deployment process is set up (eg with Jenkins or Github actions) that can deploy new versions of the service; staging and dev environments are set up for validating changes; unit and integration tests are added to the code; 
